In [6]:
import pandas as pd
import numpy as np
import json
from app_store_scraper import AppStore
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer3
from sklearn.decomposition import LatentDirichletAllocation

ImportError: cannot import name 'TfidfVectorizer3' from 'sklearn.feature_extraction.text' (/Users/marcuschandra/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py)

In [7]:
pip install app_store_scraper
pip install google-play-scraper
pip install --upgrade pip
pip install pandas spacy

SyntaxError: invalid syntax (1912247331.py, line 1)

In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))
custom_stop_words = ['recommend', 'love', 'helpful',]
stop_words.update(custom_stop_words)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
italki = AppStore(country = 'us', app_name = 'italki-language-learning', app_id = '1140000003')
italki.review(how_many = 5000)

In [4]:
italkidf = pd.DataFrame(np.array(italki.reviews), columns = ['review'])
italkidf2 = italkidf.join(pd.DataFrame(italkidf.pop('review').tolist()))
italkidf2

NameError: name 'italki' is not defined

In [19]:
italki_positive = italkidf2[italkidf2['rating']>=4]
italki_negative = italkidf2[italkidf2['rating']<=2]
len(italki_positive), len(italki_negative)

(287, 128)

In [176]:
positive_reviews_app_store = italki_positive['review'].tolist()
negative_reviews_app_store = italki_negative['review'].tolist()

In [61]:
from google_play_scraper import app
from google_play_scraper import Sort, reviews_all

gp_italki = reviews_all(
    'com.italki.app',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
) 

italki_gp_df = pd.DataFrame(np.array(gp_italki),columns=['review'])
italki_gp_df = italki_gp_df.join(pd.DataFrame(italki_gp_df.pop('review').tolist()))

In [105]:
italki_gp_positive = italki_gp_df[italki_gp_df['score']>=4]
italki_gp_negative = italki_gp_df[italki_gp_df['score']<=2]
len(italki_gp_positive), len(italki_gp_negative)

(1124, 706)

In [177]:
positive_reviews_gp = italki_gp_positive['content'].tolist()
negative_reviews_gp = italki_gp_negative['content'].tolist()
total_positive = positive_reviews_gp + positive_reviews_app_store
total_negative = negative_reviews_gp + negative_reviews_app_store

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marcuschandra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marcuschandra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [179]:
stop_words = set(stopwords.words('english'))

# Sample data
filtered_reviews = []
for review in total_positive:
    word_tokens = word_tokenize(review.lower())  # Tokenize and convert to lowercase
    filtered_review = [word for word in word_tokens if not word in stop_words and word.isalpha()]
    filtered_reviews.extend(filtered_review)

    
from nltk.probability import FreqDist
fdist = FreqDist(filtered_reviews)
for word, frequency in fdist.most_common(30):  # Adjust the number to see more or fewer common words
    print(f"{word}: {frequency}")

app: 646
italki: 407
language: 379
great: 283
teachers: 268
good: 244
learning: 240
teacher: 222
learn: 221
really: 170
lessons: 147
use: 139
easy: 138
love: 130
like: 124
one: 121
languages: 109
find: 108
best: 105
platform: 101
way: 101
lesson: 99
amazing: 91
time: 88
recommend: 87
new: 82
native: 80
get: 77
using: 73
well: 69


In [209]:
#LDA model

vectorizer = CountVectorizer(ngram_range=(1,3), stop_words=stop_words, max_df=0.1, min_df=25, max_features=1000)
review_term_matrix = vectorizer.fit_transform(total_negative)

n_topics = 20
lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
lda.fit(review_term_matrix)

feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx+1}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]]))

Topic #1:
email way get nothing without support still tried send using
Topic #2:
phone much many using make people one also great credits
Topic #3:
website never cannot send give like get teachers also could
Topic #4:
problem one get bad times help back good tried trying
Topic #5:
see lessons also account want phone problem back still give
Topic #6:
like able message would used use app way something please give
Topic #7:
teachers support times available many still useless great help always
Topic #8:
lesson bad could make every available see update new show
Topic #9:
class show still option another give would every cannot fix
Topic #10:
free good messages lessons also first still using please would
Topic #11:
want people learn good like learning trying help great could
Topic #12:
service customer really useless support great go bad credits used
Topic #13:
update need working something always account one like know still
Topic #14:
error pay try every message application please working pr

In [210]:
#NMF model

tfidf_vectorizer = TfidfVectorizer(max_df=0.1, min_df=30, stop_words=stop_words)
review_term_matrix = tfidf_vectorizer.fit_transform(total_positive)
nmf_model = NMF(n_components=15, random_state=1)  # n_components is the number of topics
nmf_topic_matrix = nmf_model.fit_transform(review_term_matrix)


feature_names = tfidf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmf_model.components_):
    print(f"Topic #{topic_idx + 1}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]]))

/Users/marcuschandra/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


Topic #1:
like would far know please lesson book time also found
Topic #2:
easy use classes way schedule affordable teaching book community application
Topic #3:
amazing much application skills tutor highly people could practice community
Topic #4:
nice people always time know could able tutor use lesson
Topic #5:
best way english teaching different spanish native know speakers improve
Topic #6:
excellent lot application spanish thanks far highly affordable way many
Topic #7:
languages different many lot world people also choose way want
Topic #8:
find tutor way needs try also different easy highly students
Topic #9:
one classes lot try get make work could people also
Topic #10:
lessons lesson much lot thanks also get book always thank
Topic #11:
platform using years students needs awesome teaching thank highly classes
Topic #12:
useful english application thank much thanks improve think help lesson
Topic #13:
works well lesson work thank better students need using spanish
Topic #14:
n

/Users/marcuschandra/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
